# Exploring a DataFrames version of the former work

## Starting PySparkSQL

This interface allows the use the tabular methods provided by DataFrames. Spark offers a DataFrame API, through their Spark SQL.

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Cluster analysis") \
    .getOrCreate()

23/01/12 10:24:52 WARN Utils: Your hostname, Kixus-k resolves to a loopback address: 127.0.1.1; using 130.190.48.180 instead (on interface wlp4s0)
23/01/12 10:24:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/12 10:24:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Parsing the schema file

In [3]:
from schema import Schema
from tabulate import tabulate
from importlib import reload

schema = Schema("../data/schema.csv")

## Loading a CSV file

To [load a CSV as a DataFrame](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameReader.csv.html#pyspark.sql.DataFrameReader.csv),

In [4]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, FloatType, IntegerType, LongType

# Mapping from field type as provided by the schema file, to their SparkSQL equivalent types
field_to_spark_type = {
    'time': LongType,
    'machine ID': LongType,
    'event type': IntegerType,
    'platform ID': StringType,
    'CPUs': FloatType,
    'Memory': FloatType
}

# Object representing the format of the data we will be loading, so SparkSQL can do the proper pre-processing
machine_events_schema = StructType([
    StructField(field['content'].replace(' ', '_'), field_to_spark_type[field['content']](), field['mandatory'])
    for field in schema.get_table_schema('machine_events')['fields']
])

machine_events = spark.read \
    .format('csv') \
    .option("header","false") \
    .schema(machine_events_schema) \
    .load("../data/machine_events/part-00000-of-00001.csv")

Let us check that the schema is as we expect

In [5]:
machine_events.printSchema()

root
 |-- time: long (nullable = true)
 |-- machine_ID: long (nullable = true)
 |-- event_type: integer (nullable = true)
 |-- platform_ID: string (nullable = true)
 |-- CPUs: float (nullable = true)
 |-- Memory: float (nullable = true)



And inspect the first few data

In [6]:
for elem in machine_events.take(5):
	print(elem.asDict())

{'time': 0, 'machine_ID': 5, 'event_type': 0, 'platform_ID': 'HofLGzk1Or/8Ildj2+Lqv0UGGvY82NLoni8+J/Yy0RU=', 'CPUs': 0.5, 'Memory': 0.2493000030517578}
{'time': 0, 'machine_ID': 6, 'event_type': 0, 'platform_ID': 'HofLGzk1Or/8Ildj2+Lqv0UGGvY82NLoni8+J/Yy0RU=', 'CPUs': 0.5, 'Memory': 0.2493000030517578}
{'time': 0, 'machine_ID': 7, 'event_type': 0, 'platform_ID': 'HofLGzk1Or/8Ildj2+Lqv0UGGvY82NLoni8+J/Yy0RU=', 'CPUs': 0.5, 'Memory': 0.2493000030517578}
{'time': 0, 'machine_ID': 10, 'event_type': 0, 'platform_ID': 'HofLGzk1Or/8Ildj2+Lqv0UGGvY82NLoni8+J/Yy0RU=', 'CPUs': 0.5, 'Memory': 0.2493000030517578}
{'time': 0, 'machine_ID': 13, 'event_type': 0, 'platform_ID': 'HofLGzk1Or/8Ildj2+Lqv0UGGvY82NLoni8+J/Yy0RU=', 'CPUs': 0.5, 'Memory': 0.2493000030517578}


How much events do we have?

In [7]:
machine_events.count(), machine_events.filter(machine_events.event_type == 1).count(),machine_events.filter(machine_events.event_type == 2).count()

(37780, 8957, 7380)

In [8]:
creation_events = machine_events.filter(machine_events.event_type == 0)

cpu_counts = creation_events.groupBy("CPUs").count()
cpu_counts.show()

+----+-----+
|CPUs|count|
+----+-----+
|0.25|  224|
|null|   32|
| 0.5|19773|
| 1.0| 1414|
+----+-----+



Just because we can, here is a version in SQL:

In [9]:
creation_events.createOrReplaceTempView("creation_events")

spark.sql("SELECT CPUs, COUNT(*) "
           "FROM creation_events "
           "GROUP BY CPUs;").show()

+----+--------+
|CPUs|count(1)|
+----+--------+
|0.25|     224|
|null|      32|
| 0.5|   19773|
| 1.0|    1414|
+----+--------+



## Percentage of lost computational power

In [10]:
machine_events.createOrReplaceTempView("machine_events")

In [11]:
bad_cpu_machines = spark.sql("SELECT DISTINCT machine_ID "
                             "FROM machine_events "
                             "WHERE CPUs IS NULL")
bad_cpu_machines.count()

32

In [12]:
spark.sql(
    "SELECT * FROM machine_events "
    "WHERE CPUs IS NOT NULL "
    # "GROUP BY machine_ID "
    "ORDER BY time "
    ).show()

+----+----------+----------+--------------------+----+------+
|time|machine_ID|event_type|         platform_ID|CPUs|Memory|
+----+----------+----------+--------------------+----+------+
|   0|         5|         0|HofLGzk1Or/8Ildj2...| 0.5|0.2493|
|   0|         6|         0|HofLGzk1Or/8Ildj2...| 0.5|0.2493|
|   0|         7|         0|HofLGzk1Or/8Ildj2...| 0.5|0.2493|
|   0|        10|         0|HofLGzk1Or/8Ildj2...| 0.5|0.2493|
|   0|        13|         0|HofLGzk1Or/8Ildj2...| 0.5|0.2493|
|   0|        14|         0|HofLGzk1Or/8Ildj2...| 0.5|0.2493|
|   0|        19|         0|HofLGzk1Or/8Ildj2...| 0.5|0.2493|
|   0|        21|         0|HofLGzk1Or/8Ildj2...| 0.5|0.2493|
|   0|        23|         0|HofLGzk1Or/8Ildj2...| 0.5|0.2493|
|   0|        25|         0|HofLGzk1Or/8Ildj2...| 0.5|0.2493|
|   0|        26|         0|HofLGzk1Or/8Ildj2...| 0.5|0.2493|
|   0|        27|         0|HofLGzk1Or/8Ildj2...| 0.5|0.2493|
|   0|        28|         0|HofLGzk1Or/8Ildj2...| 0.5|0.2493|
|   0|  

Let us check the maximum number of events associated with any machine:

In [13]:
spark.sql(
    "SELECT COUNT(*) as n_events FROM machine_events "
    "WHERE CPUs IS NOT NULL "
    "GROUP BY machine_ID "
    "ORDER BY n_events DESC "
    "LIMIT 1"
    ).show()

+--------+
|n_events|
+--------+
|     330|
+--------+



We have here 330 events at most for a machine. It should be easily managable by regular python code so we can proceed with the approach below.

In [14]:
from pyspark.sql import functions as F
from pyspark.sql.functions import *

grouped_machine_events = machine_events \
    .filter(machine_events['CPUs'].isNotNull()) \
    .orderBy(machine_events['time']) \
    .groupBy(machine_events['machine_ID']) \
    .agg(
        F.collect_list('time').alias('time'),
        F.collect_list('event_type').alias('event_type'),
        F.collect_list('CPUs').alias('CPUs'))

In [15]:
grouped_machine_events.show()

+----------+--------------------+--------------------+--------------------+
|machine_ID|                time|          event_type|                CPUs|
+----------+--------------------+--------------------+--------------------+
|         5|[835150655707, 83...|           [1, 0, 0]|     [0.5, 0.5, 0.5]|
|         6|                 [0]|                 [0]|               [0.5]|
|         7|                 [0]|                 [0]|               [0.5]|
|        10|[1306164355566, 1...|           [1, 0, 0]|     [0.5, 0.5, 0.5]|
|        13|[2447693838527, 2...|           [1, 0, 0]|     [0.5, 0.5, 0.5]|
|        14|                 [0]|                 [0]|               [0.5]|
|        19|                 [0]|                 [0]|               [0.5]|
|        21|[1973117638055, 1...|        [2, 2, 2, 0]|[0.5, 0.5, 0.5, 0.5]|
|        23|[481673206369, 48...|     [2, 2, 1, 0, 0]|[0.5, 0.5, 0.5, 0...|
|        25|[669036455186, 68...|[2, 2, 2, 2, 2, 2...|[0.5, 0.5, 0.5, 0...|
|        26|

In [ ]:
from pyspark.sql import Column

def col_gen(df):
    print(len(F.array(df.time)))
    # accumulateur = df.machine_ID * 0
    # for t in  df.time:
    #     accumulateur += t
    return df.time

grouped_machine_events.withColumn('aggrégée', col_gen(grouped_machine_events)).show()

TypeError: object of type 'Column' has no len()

In [ ]:
from pyspark.sql.functions import pandas_udf

def filter_func(iterator):
    for pdf in iterator:
        print(type(pdf))
        yield pdf

grouped_machine_events.mapInPandas(filter_func, grouped_machine_events.schema).show()

+----------+--------------------+--------------------+--------------------+
|machine_ID|                time|          event_type|                CPUs|
+----------+--------------------+--------------------+--------------------+
|         5|[835150655707, 83...|           [1, 0, 0]|     [0.5, 0.5, 0.5]|
|         6|                 [0]|                 [0]|               [0.5]|
|         7|                 [0]|                 [0]|               [0.5]|
|        10|[1306164355566, 1...|           [1, 0, 0]|     [0.5, 0.5, 0.5]|
|        13|[2447693838527, 2...|           [1, 0, 0]|     [0.5, 0.5, 0.5]|
|        14|                 [0]|                 [0]|               [0.5]|
|        19|                 [0]|                 [0]|               [0.5]|
|        21|[1973117638055, 1...|        [2, 2, 2, 0]|[0.5, 0.5, 0.5, 0.5]|
|        23|[481673206369, 48...|     [2, 2, 1, 0, 0]|[0.5, 0.5, 0.5, 0...|
|        25|[669036455186, 68...|[2, 2, 2, 2, 2, 2...|[0.5, 0.5, 0.5, 0...|
|        26|

<class 'pandas.core.frame.DataFrame'>                               (0 + 1) / 1]
<class 'pandas.core.frame.DataFrame'>


In [ ]:
def my_function(df):
    df.time[0]+1

grouped_machine_events.to_pandas_on_spark().apply(my_function)

/home/dorian/Logiciels/spark-3.3.0-bin-hadoop3/python/pyspark/sql/dataframe.py:3313: FutureWarning: DataFrame.to_pandas_on_spark is deprecated. Use DataFrame.pandas_api instead.
  warnings.warn(
/home/dorian/Logiciels/spark-3.3.0-bin-hadoop3/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If the type hints is not specified for `apply`, it is expensive to infer the data type internally.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)
ERROR:root:KeyboardInterrupt while sending command.                 (0 + 1) / 1]
Traceback (most recent call last):
  File "/home/dorian/Logiciels/spark-3.3.0-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/dorian/Logiciels/spark-3.3.0-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", li

KeyboardInterrupt: 

23/01/12 10:23:56 ERROR TransportChannelHandler: Connection to eduroam-049081.grenet.fr/130.190.49.81:37819 has been quiet for 120000 ms while there are outstanding requests. Assuming connection is dead; please adjust spark.shuffle.io.connectionTimeout if this is wrong.
23/01/12 10:23:56 ERROR TransportResponseHandler: Still have 1 requests outstanding when connection from eduroam-049081.grenet.fr/130.190.49.81:37819 is closed
23/01/12 10:23:56 ERROR OneForOneBlockFetcher: Failed while starting block fetches
java.io.IOException: Connection from eduroam-049081.grenet.fr/130.190.49.81:37819 closed
	at org.apache.spark.network.client.TransportResponseHandler.channelInactive(TransportResponseHandler.java:147)
	at org.apache.spark.network.server.TransportChannelHandler.channelInactive(TransportChannelHandler.java:117)
	at io.netty.channel.AbstractChannelHandlerContext.invokeChannelInactive(AbstractChannelHandlerContext.java:262)
	at io.netty.channel.AbstractChannelHandlerContext.invokeChann

In [ ]:
import pandas as pd
from pyspark.sql.functions import pandas_udf

In [ ]:
event_mapping = {0: 'add', 1: 'remove', 2: 'update'}

@pandas_udf("cpu double")
def lost_and_total_power(time: pd.Series, event_type: pd.Series, CPUs: pd.Series) -> tuple[float, float]:
    return CPUs[-1]
    min_cpu = float('inf')
    max_cpu = float('-inf')
    for i, ev in events.items():
        # event_to_accumulator(event)
        # update_accumulator(event)
        if ev['CPUs'] < min_cpu:
            min_cpu = ev['CPUs']
        if ev['CPUs'] > max_cpu:
            max_cpu = ev['CPUs']
    return pd.DataFrame({'min cpu': min_cpu, 'max cpu': max_cpu})


grouped_machine_events \
    .agg(
        lost_and_total_power(
            machine_events['time'],
            machine_events['event_type'],
            machine_events['CPUs'])) \
    .show()

NotImplementedError: Invalid  return type with grouped aggregate Pandas UDFs: StructType([StructField('cpu', DoubleType(), True)]) is not supported

This route seams to be a dead end. The `applyInPandas` method on grouped data seems to not allow for iterative code. How can I do this aggregation then ?

Based on the documentation[[1]](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.pandas_udf.html#pyspark.sql.functions.pandas_udf)[[2]](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.GroupedData.applyInPandas.html#pyspark.sql.GroupedData.applyInPandas) and our experiments, it seems infeasible to use the grouped data to compute the lost computational power.

## In general, do tasks from the same job run on the same machine?

In [ ]:
In general, do tasks from the same job run on the same machine?